# <font color='blue'>Data Science Academy</font>
# <font color='blue'>Análise Estatística Para Data Science com Linguagem Python</font>

## <font color='blue'>Estudo de Caso 2</font>
### <font color='blue'>Prevendo o Rendimento de Fundos Imobiliários em Meio a Incertezas com Regressão Bayesiana</font>

![DSA](imagens/EC2.png)

## Instalando e Carregando os Pacotes

In [1]:
# Versão da Linguagem Python
from platform import python_version
print('Versão da Linguagem Python Usada Neste Jupyter Notebook:', python_version())

Versão da Linguagem Python Usada Neste Jupyter Notebook: 3.9.18


In [ ]:
# Para atualizar um pacote, execute o comando abaixo no terminal ou prompt de comando:
# pip install -U nome_pacote

# Para instalar a versão exata de um pacote, execute o comando abaixo no terminal ou prompt de comando:
# !pip install nome_pacote==versão_desejada

# Depois de instalar ou atualizar o pacote, reinicie o jupyter notebook.

# Instala o pacote watermark. 
# Esse pacote é usado para gravar as versões de outros pacotes usados neste jupyter notebook.
# !pip install -q -U watermark

In [3]:
!pip3 install -q scipy==1.7.3


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


https://www.pymc.io/projects/docs/en/v3/index.html

In [4]:
!pip3 install -q pymc3==3.11.5

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gluonts 0.13.2 requires pydantic~=1.7, but you have pydantic 2.5.3 which is incompatible.
skimpy 0.0.14 requires numpy<2.0.0,>=1.22.2, but you have numpy 1.22.1 which is incompatible.

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [5]:
# Imports
import pandas as pd
import numpy as np
import pymc3 as pm
import warnings
warnings.filterwarnings('ignore')

In [6]:
# Versões dos pacotes usados neste jupyter notebook
%reload_ext watermark
%watermark -a "Data Science Academy" --iversions

Author: Data Science Academy

pandas: 2.0.3
pymc3 : 3.11.5
numpy : 1.22.1



## Carregando os Dados

In [7]:
# Carregar o dataset de investimentos
dados_investimentos = pd.read_csv('dados/dados_investimentos.csv', index_col = 0, parse_dates = True )

In [8]:
dados_investimentos.shape

(365, 6)

In [9]:
dados_investimentos.head()

,Ações,Títulos Públicos,Fundos Imobiliários,Rendimento Ações,Rendimento Títulos Públicos,Rendimento Fundos Imobiliários
2022-01-01,-0.017334,-0.048899,0.510967,NaN,NaN,NaN
2022-01-02,0.435083,-0.159320,-0.189749,-26.099910,2.258122,-1.371353
2022-01-03,0.196220,-0.101227,0.159899,-0.549005,-0.364632,-1.842684
2022-01-04,0.023019,0.072812,-0.402519,-0.882688,-1.719296,-3.517339
2022-01-05,0.103289,0.136370,0.192497,3.487111,0.872901,-1.478231


In [11]:
# Selecionar apenas os dados de rendimento dos fundos imobiliários
dados_fundos_imobiliarios = dados_investimentos[['Rendimento Fundos Imobiliários']]

In [12]:
dados_fundos_imobiliarios.shape

(365, 1)

In [13]:
dados_fundos_imobiliarios.head()

,Rendimento Fundos Imobiliários
2022-01-01,NaN
2022-01-02,-1.371353
2022-01-03,-1.842684
2022-01-04,-3.517339
2022-01-05,-1.478231


## Pré-Processamento dos Dados

In [14]:
# Criar a variável preditora
dias_deslocamento = 5
dados_fundos_imobiliarios['Rendimento Fundos Imobiliários (t-5)'] = dados_fundos_imobiliarios['Rendimento Fundos Imobiliários'].shift(dias_deslocamento)

In [15]:
dados_fundos_imobiliarios.head()

,Rendimento Fundos Imobiliários,Rendimento Fundos Imobiliários (t-5)
2022-01-01,NaN,NaN
2022-01-02,-1.371353,NaN
2022-01-03,-1.842684,NaN
2022-01-04,-3.517339,NaN
2022-01-05,-1.478231,NaN


In [16]:
dados_fundos_imobiliarios.tail()

,Rendimento Fundos Imobiliários,Rendimento Fundos Imobiliários (t-5)
2022-12-27,-0.104117,-4.930165
2022-12-28,-1.485806,-1.328787
2022-12-29,0.113994,-1.616446
2022-12-30,-2.652999,-2.591367
2022-12-31,-1.680685,-3.610346


In [17]:
# Remover as linhas com valores ausentes
dados_fundos_imobiliarios = dados_fundos_imobiliarios.dropna()

In [18]:
dados_fundos_imobiliarios.head()

,Rendimento Fundos Imobiliários,Rendimento Fundos Imobiliários (t-5)
2022-01-07,-3.025357,-1.371353
2022-01-08,-1.948727,-1.842684
2022-01-09,-0.055173,-3.517339
2022-01-10,-1.490320,-1.478231
2022-01-11,-3.540331,-0.196477


In [19]:
# Separar os dados em conjunto de treinamento e teste
dados_treino = dados_fundos_imobiliarios.loc['2022-01-06':'2022-10-31']
dados_teste = dados_fundos_imobiliarios.loc['2022-11-01':'2022-12-31']

## Construção e Treinamento do Modelo

Diferentemente da regressão clássica, que assume que os parâmetros do modelo são desconhecidos, a Regressão Bayesiana modela esses parâmetros como distribuições de probabilidade.

Na Regressão Bayesiana, os dados observados são usados para atualizar a distribuição a priori dos parâmetros do modelo, resultando em uma distribuição a posteriori. Essa distribuição a posteriori é então usada para fazer previsões sobre novos dados.

A Regressão Bayesiana é útil quando há incerteza sobre os parâmetros do modelo ou quando há poucos dados disponíveis. Ela permite incorporar informações prévias (priors) sobre os parâmetros do modelo, o que pode melhorar a precisão das previsões. Além disso, a Regressão Bayesiana fornece uma distribuição completa dos parâmetros do modelo, em vez de apenas um único valor pontual, permitindo uma avaliação mais completa da incerteza.

O código abaixo usa a biblioteca PyMC3 para modelar uma regressão linear bayesiana.

**with pm.Model() as modelo:** inicia um novo modelo usando o PyMC3.

**alpha = pm.Normal('alpha', mu = 0, sd = 1)** define a priori para o intercepto alpha como uma distribuição normal com média 0 e desvio padrão 1.

**beta = pm.Normal('beta', mu = 0, sd = 1)** define a priori para o coeficiente de regressão beta como uma distribuição normal com média 0 e desvio padrão 1.

**sigma = pm.HalfNormal('sigma', sd = 1)** define a priori para o desvio padrão sigma do erro como uma distribuição normal com média 0 e desvio padrão 1.

**mu = alpha + beta*dados_treino['Rendimento Fundos Imobiliários (t-5)']** define a média do modelo como uma combinação linear do intercepto e coeficiente multiplicado pela variável preditora dados_treino['Rendimento Fundos Imobiliários (t-5)'].

**likelihood = pm.Normal('y', mu = mu, sd = sigma, observed = dados_treino['Rendimento Fundos Imobiliários'])** define a verossimilhança do modelo, que é uma distribuição normal com média mu e desvio padrão sigma, observada nos dados dados_treino['Rendimento Fundos Imobiliários'].

**trace = pm.sample(1000, tune = 1000)** executa a amostragem de Monte Carlo usando a função sample do PyMC3, que gera 1000 amostras a partir do modelo especificado, depois de 1000 iterações de tuning para ajustar os hiperparâmetros do algoritmo. O resultado é armazenado na variável trace.

In [20]:
# Criar o modelo de regressão linear bayesiana

# Inicia um novo modelo usando o PyMC3.
with pm.Model() as modelo:    
    
    # priors
    alpha = pm.Normal('alpha', mu = 0, sd = 1)
    beta = pm.Normal('beta', mu = 0, sd = 1)
    sigma = pm.HalfNormal('sigma', sd = 1)

    # likelihood
    mu = alpha + beta * dados_treino['Rendimento Fundos Imobiliários (t-5)']
    likelihood = pm.Normal('y', mu = mu, sd = sigma, observed = dados_treino['Rendimento Fundos Imobiliários'])

    # inferência
    trace = pm.sample(1000, tune = 1000)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [sigma, beta, alpha]


/Users/frnepom/.pyenv/versions/3.9.18/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
/Users/frnepom/.pyenv/versions/3.9.18/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
/Users/frnepom/.pyenv/versions/3.9.18/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
/Users/frnepom/.pyenv/versions/3.9.18/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 7 seconds.


No exemplo acima, as distribuições pm.Normal e pm.HalfNormal definem as priors (a priori) para os parâmetros do modelo bayesiano.

As priors são distribuições de probabilidade que expressam as crenças iniciais (ou incertezas) sobre os valores dos parâmetros antes de observar os dados.

No caso do código acima, as priors definem as crenças iniciais sobre os valores dos parâmetros alpha, beta e sigma. A distribuição normal com média zero e desvio padrão um, por exemplo, expressa a crença inicial de que o valor de alpha está mais provavelmente próximo a zero, mas pode ser positivo ou negativo, e é tão provável que seja 1 desvio padrão acima ou abaixo de zero.

As priors podem ser escolhidas com base no conhecimento prévio ou na opinião do modelador, ou com base em informações adicionais, como estudos anteriores, teoria, experimentos piloto ou dados históricos.

## Uso do Modelo com Dados de Teste

A linha de código abaixo usa o PyMC3 para gerar amostras preditivas do modelo bayesiano já treinado usando as amostras da posteriori armazenadas na variável trace.

Em particular, a função pm.sample_posterior_predictive é usada para gerar previsões do modelo usando as amostras da posteriori. Os argumentos samples, model e var_names são usados para especificar o número de amostras a serem geradas, o modelo a ser usado e as variáveis de interesse para as quais se deseja prever.

A linha ['y'].mean(axis=0) significa que estamos interessados em prever a variável y, que é a variável de resposta no modelo, e que estamos calculando a média das previsões para cada ponto no conjunto de dados de teste.

Portanto, a variável rendimento_previsto conterá a previsão média do modelo para a variável y no conjunto de dados de teste, com base nas amostras preditivas geradas a partir da posteriori.

In [21]:
# Fazer previsões para o conjunto de teste
rendimento_previsto = pm.sample_posterior_predictive(trace,
                                                     samples = 1000,
                                                     model = modelo,
                                                     var_names = ['alpha', 'beta', 'sigma', 'y'])['y'].mean(axis=0)

In [22]:
rendimento_previsto

array([-0.41509814, -0.4074038 ,  0.12426071, -0.15115973, -0.2045961 ,
        0.54361488,  0.49358108,  0.55306589,  0.23121378,  0.32303125,
        0.82377935,  0.40430152,  0.31405272,  0.26046138, -0.46545189,
        0.34376238, -0.18117659,  0.14691323,  0.84873968,  0.66414986,
        0.62002433,  0.16212076,  0.47407428, -0.41978618, -0.7415603 ,
       -0.25793988,  0.29503383, -0.67036825,  0.98114276, -0.03398677,
       -0.44048356, -0.15975451, -0.23341828,  0.14509498,  0.67985712,
        0.77971618, -0.00993965,  0.32530401,  0.45744679,  1.20443868,
        0.24176794, -0.08141043,  0.40762408,  0.02723157,  0.31147192,
        0.21307349, -0.24857222,  2.0880174 , -0.16792487,  0.60642531,
       -0.04935323,  0.25213487,  0.65978766, -0.04118608,  0.71698129,
        0.70126325, -0.22056108,  0.43198087,  0.26444931,  0.49101008,
        1.37458976,  0.51846174,  0.49180625,  0.02014964,  0.24675342,
       -0.33319491, -0.76353708,  0.29251712, -0.23507396, -0.03

## Previsões com Novos Dados

In [23]:
# Carregar os novos dados de entrada
novos_dados = pd.read_csv('dados/novos_dados.csv', index_col = 0, parse_dates = True)

In [24]:
# Selecionar apenas a variável preditora
novos_dados_preditor = novos_dados['Rendimento Fundos Imobiliários'].shift(5)

In [25]:
# Remover as linhas com valores ausentes
novos_dados_preditor = novos_dados_preditor.dropna()

In [26]:
# Fazer as previsões para os novos dados de entrada
with modelo:
    alpha = trace['alpha'].mean()
    beta = trace['beta'].mean()
    
    previsoes = alpha + beta * novos_dados_preditor

In [27]:
print(previsoes)

2023-01-07    0.241797
2023-01-08    0.244685
2023-01-09    0.247334
2023-01-10    0.366980
2023-01-11    0.241963
                ...   
2023-12-27    0.236748
2023-12-28    0.237088
2023-12-29    0.247070
2023-12-30    0.301834
2023-12-31    0.241352
Name: Rendimento Fundos Imobiliários, Length: 359, dtype: float64


# Fim